# 1. Вытаскиваем ссылки, которые накачали

In [1]:
import requests
from bs4 import BeautifulSoup
from requests.exceptions import ConnectionError

# библиотека для создания юзерагентов
# from fake_useragent import UserAgent

from tqdm import tqdm_notebook
import re
import pickle
import time

from cian_info import *

In [2]:
with open('hrefs_total.pickle', 'rb') as f:
    hrefs_total = pickle.load(f)
    
len(hrefs_total)

73924

In [3]:
type(hrefs_total), len(set(hrefs_total))

(list, 73924)

In [4]:
url = "https://www.cian.ru/sale/flat/225645762/"

In [ ]:
get_soup(url)

### Чтобы включать ТОР

In [ ]:
import socks
import socket
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

In [ ]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = BeautifulSoup(ip, 'html.parser')
    print(soup.find('body').text)

checkIP()

# 2. Качаем основную информацию по квартирам

Объединяем всё в одну функцию! 

In [5]:
def infa_download(soup):
    infa_dict = {}
    infa_dict.update(get_general_information(soup))
    infa_dict.update(get_about(soup))
    infa_dict.update(get_price(soup))
    infa_dict.update(get_description(soup))
    infa_dict.update(get_square_info(soup))
    infa_dict.update(get_address(soup))
    infa_dict.update(get_metro(soup))
    infa_dict.update(get_title(soup))
    infa_dict.update(get_coord(soup))
    infa_dict.update(get_reinovation(soup))
    return infa_dict

Основной цикл: 

In [6]:
all_infa = [ ] # для данных

In [ ]:
for url in tqdm_notebook(hrefs_total[10001:25001]):
    print(url)
    
    try:
        time.sleep(10)
        soup = get_soup(url)
        
    except ConnectionError:
        print('конэкшон ерор')
        time.sleep(10)
        soup = get_soup(url)
        
    # пока вылезает капча, отдыхаем
    while is_capcha_or_badip(soup):
        time.sleep(60*10)            # сплю N секунд
        soup = get_soup(url)     # внеочередная попытка пробится сквозь капчу 
            
    if soup.find("div", {"data-name": "OfferUnpublished"}):
        print('объявление снято')
        continue
        
    cur_infa = infa_download(soup)
    cur_infa['href'] = url
    all_infa.append(cur_infa)

F:\Conda\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


https://www.cian.ru/sale/flat/223258344/
Капча вылезла :(


In [ ]:
hrefs_total.index(url)  # чтобы понимать откуда продолжать скачку, если код упал

In [ ]:
len(all_infa)

Сохранялка. Сохраняем в формате `info_2000_5000.pickle`, где цифры означают с какого по какой файл скачали.

In [ ]:
# осторожнее с именами файлов! случайно можно перезаписать готовый :( 
with open('data/info_10001_25000.pickle', 'wb') as f:
    pickle.dump(all_infa, f)

In [ ]:
# можно проверить всё ли ок сохранилось
with open('data/info_0_10.pickle', 'rb') as f:
    data_new = pickle.load(f)

Поглазеть на данные: 

In [ ]:
import pandas as pd
df = pd.DataFrame(all_infa)
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()